In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action="ignore")

titanic_df = pd.read_csv("titanic_train.csv")
titanic_df.head()
titanic_df.dropna(inplace=True)
titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [27]:
titanic_df.dropna(inplace=True)
titanic_df["Sex"] = titanic_df.Sex.replace({'male':0,'female':1})
titanic_df["Embarked"] = titanic_df.Embarked.replace({'S':1,'C':2,'Q':3})
y = titanic_df['Survived']
X = titanic_df[['Sex','Age','SibSp','Parch','Embarked']]

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 40)

In [71]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 20)
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=20)

In [72]:
knn.score(X_test, y_test)

0.7027027027027027

In [78]:
from sklearn.model_selection import cross_val_score

cvscores_3 = cross_val_score(knn, X_test, y_test, cv=8)
print('cv =  3 ---> Ortalama çarpraz doğrulama skoru : {:.2f} '.format(np.mean(cvscores_3)))

cvscores_10 = cross_val_score(knn, X_test, y_test, cv=10)
print('cv = 10 ---> Ortalama çarpraz doğrulama skoru : {:.2f} '.format(np.mean(cvscores_10)))

cvscores_11 = cross_val_score(knn, X_test, y_test, cv=11)
print('cv = 11 ---> Ortalama çarpraz doğrulama skoru : {:.2f} '.format(np.mean(cvscores_11)))


cv =  3 ---> Ortalama çarpraz doğrulama skoru : 0.71 
cv = 10 ---> Ortalama çarpraz doğrulama skoru : 0.70 
cv = 11 ---> Ortalama çarpraz doğrulama skoru : 0.70 


**cv'yi 11 e çıkardığımızda doğruluk oranımız %70'den %71'e çıktı.**

**Ağırlıklandırma ile de deneyelim.**

In [32]:
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats

neighbors = KNeighborsClassifier(n_neighbors=5, weights='distance')

X_zscore = stats.zscore(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_zscore, y, test_size=0.2, random_state = 40)

knn.fit(X_train,y_train)
knn.score(X_test, y_test)

0.5135135135135135

**Modelin performansı düştü.**

# Regresyon ile deneyelim.

In [34]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse

import warnings
warnings.filterwarnings('ignore')
house_prices_df = pd.read_csv("house_prices_train.csv")
house_prices_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [38]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=10)

new_features = house_prices_df[["YearBuilt","LotArea","TotRmsAbvGrd","FullBath","MoSold","HalfBath","GarageCars","SalePrice"]]
Y = new_features['SalePrice']
X = new_features[['YearBuilt','LotArea','MoSold','TotRmsAbvGrd','FullBath','HalfBath','GarageCars']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_zscore, y, test_size=0.2, random_state = 40)

knn.fit(X_train,y_train)
knn.score(X_test, y_test)

0.05818181818181778

In [44]:
knn.predict(X_test)

array([1. , 0.4, 1. , 0.6, 0.9, 0.6, 0.6, 0.3, 0.5, 0.4, 0.3, 1. , 0.6,
       0.8, 0.9, 0.6, 0.8, 0.9, 0.5, 0.6, 0.5, 0.9, 0.8, 0.5, 0.3, 0.9,
       0.9, 0.4, 0.5, 0.9, 1. , 1. , 0.6, 0.6, 1. , 0.6, 0.5])